In [ ]:
!pip install --upgrade google-cloud-vision
!pip install utils

In [ ]:
import os
import io
import numpy as np
import pandas as pd
import platform
from PIL import ImageFont, ImageDraw, Image
import utils
import matplotlib.pyplot as plt
import cv2
from google.cloud import vision
from google.cloud import vision_v1

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ultra-airway-412008-69f1d43b4fb7.json'

# client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
# client = vision.ImageAnnotatorClient(client_options=client_options)
client_options = {'api_endpoint': 'vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [ ]:
df = pd.read_csv('data2.csv',encoding = 'utf-8')
df=df.drop(columns = ['Unnamed: 0','level_0','index'])

In [ ]:
image_folder_path = '/content/im'
image_files = os.listdir(image_folder_path)

In [ ]:
def ocr_with_google_cloud_vision(path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    for text in texts:
        vertices = text.bounding_poly.vertices
        if len(vertices) == 4:
            x1, y1 = vertices[0].x, vertices[0].y
            x2, y2 = vertices[2].x, vertices[2].y

            height = abs(y2 - y1)
            area = abs((x2 - x1) * (y2 - y1))

            if height >= 47:
                return text.description.replace('\n', ' ')

    return ''

In [ ]:
# 유해성 검출 함수
def detect_safe_search(path):


    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)



    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    likelihood_name = (0,10,20,30,40,50)


    # score = likelihood_name[safe.adult]+likelihood_name[safe.medical]+likelihood_name[safe.spoof]+likelihood_name[safe.violence]+likelihood_name[safe.racy]


    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return likelihood_name[safe.adult], likelihood_name[safe.medical], likelihood_name[safe.spoof],likelihood_name[safe.violence],likelihood_name[safe.racy]

In [ ]:
# 제일 많은 색상의 RGB 비율 함수
def detect_properties(path):
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    fraction = []
    max = 0
    cnt = 0
    for color in props.dominant_colors.colors:
        fraction.append([color.pixel_fraction,color.color.red,color.color.green,color.color.blue])
    for idx,i in enumerate(fraction):
        if max < fraction[idx][0]:
            max = fraction[idx][0]
            cnt = idx
    # print(fraction[cnt])
    # print(props)
    # print(fraction)
    # print(fraction[cnt])



    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return fraction[cnt]

In [ ]:
# 썸네일 텍스트 위치 검출 함수
def process_image_with_text(path):
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    img = cv2.imread(path)
    roi_img = img.copy()
    height = img.shape[0]
    height1 = height * (1 / 3)
    height2 = height * (2 / 3)
    cnt = {
        'up': 0, 'middle': 0, 'down': 0,'none':0
    }

    if not texts:
        cnt['none']+=1
    else:
        for text in texts[1:-1]:


            vertices = (['({},{})'.format(vertex.x, vertex.y)
                         for vertex in text.bounding_poly.vertices])

            ocr_text = text.description
            y1 = text.bounding_poly.vertices[0].y
            y2 = text.bounding_poly.vertices[2].y

            m_y = (abs(y2 + y1)) / 2

            if 0 <= m_y and m_y < height1:
                cnt['up'] += len(ocr_text)
            elif height1 <= m_y and m_y < height2:
                cnt['middle'] += len(ocr_text)
            elif height2 <= m_y:
                cnt['down'] += len(ocr_text)
        if response.error.message:
            raise Exception(
                '{}\nFor more info on error messages, check: '
                'https://cloud.google.com/apis/design/errors'.format(
                    response.error.message))
    max_key = max(cnt, key=cnt.get)
    return max_key



In [ ]:
# 얼굴로 사람수 인식 함수
def detect_faces_uri(path):


    client = vision.ImageAnnotatorClient()

    # 이미지 파일을 읽어옵니다.
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    # 이미지 데이터를 Vision API로 전송합니다.
    image = vision.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return len(faces)

In [ ]:
# 일정 높이 이상일때 텍스트가 썸네일을 차지하는 비율 함수
def calculate_char_heights_with_text(path):
    char_info = []
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    for text in texts:
        vertices = text.bounding_poly.vertices
        if len(vertices) == 4:
            x1, y1 = vertices[0].x, vertices[0].y
            x2, y2 = vertices[2].x, vertices[2].y

            height = abs(y2 - y1)
            area = abs((x2 - x1) * (y2 - y1))

            if height >= 47:
                char_info.append(area)

    char_info.remove(char_info[0])
    return char_info

In [ ]:
# 이미지 파일 정렬함수
def sort_key(file_name):
    return int(file_name.split('_')[1].split('.')[0])
# image_files.remove('.ipynb_checkpoints')                  # 이미지 폴더 길이 확인해보고 이상한거 있으면 미리 제거
image_files_sorted = sorted(image_files, key=sort_key)

In [ ]:
for idx, image_file in enumerate(image_files_sorted):
    image_path = os.path.join(image_folder_path, image_file)


    print(image_file)
    try:
        df.loc[idx, '썸네일 문자'] = ocr_with_google_cloud_vision(image_path)

        x1, x2, x3, x4, x5 = detect_safe_search(image_path)
        df.loc[idx, 'Adult'] = x1
        df.loc[idx, 'Spoof'] = x2
        df.loc[idx, 'Medical'] = x3
        df.loc[idx, 'Violence'] = x4
        df.loc[idx,'Racy'] = x5
        properties = detect_properties(image_path)
        df.loc[idx, 'R'] = properties[1]
        df.loc[idx, 'G'] = properties[2]
        df.loc[idx, 'B'] = properties[3]
        df.loc[idx, '썸네일 텍스트 위치'] = process_image_with_text(image_path)
        df.loc[idx, '썸네일 사람수'] = detect_faces_uri(image_path)
        char_info_list = calculate_char_heights_with_text(image_path)
        total_area = sum(char_info for char_info in char_info_list[1:])
        portion = (total_area / 921600) * 100
        df.loc[idx, '썸네일 텍스트 비율'] = portion
        print(idx)
    except Exception as e:
        print(f"Error processing image {image_path} (idx: {idx}): {str(e)}")
        continue


In [ ]:
df.head()

In [ ]:
df.to_csv('',encoding="utf-8-sig")